# Interactive Video Processing with OpenCV

This Python script leverages OpenCV to create an interactive video processing application that combines live camera feed with image manipulation techniques. It processes two input images (`Do_It.jpg` and `Skyscraper.jpg`) and applies various OpenCV functionalities, displaying the results in multiple windows. The script is designed for educational purposes, demonstrating a wide range of computer vision techniques.

## Features

- **Live Video Capture**: Captures video from the default camera with customizable resolution and brightness.
- **Mouse Interaction**: Allows drawing shapes (circles, triangles, lines) on the video feed based on mouse events.
- **Trackbar Controls**: Provides adjustable parameters for HSV color detection, blending alpha, and rotation angle.
- **Multiple Display Windows**:
  - **Video Feed**: Shows the live feed with overlaid shapes, text, and object tracking.
  - **Sky Effects**: Displays edge detection and color inversion effects on `Do_It.jpg`.
  - **Skyscraper Effects**: Applies blur and contrast adjustments to `Skyscraper.jpg`.
  - **Bitwise Operations**: Visualizes AND, OR, XOR, and NOT operations between resized images.
- **Image Processing**: Includes channel splitting/merging, resizing, weighted blending, and region of interest (ROI) overlay.
- **Object Detection**: Uses HSV color space to detect and track objects in the video feed.
- **Video Recording**: Saves the processed video feed to `output_video.avi`.

## Requirements

- Python 3.x
- OpenCV (`pip install opencv-python`)
- NumPy (`pip install numpy`)

## Usage

1. Ensure `Do_It.jpg` and `Skyscraper.jpg` are in the same directory as the script.
2. Run the script:
   ```bash
   python interactive_video_processor.py
   ```
3. Interact with the application:
   - **Mouse Actions**:
     - Double left-click to draw a red circle.
     - Right-click to draw a green triangle.
     - Hold left-click and drag to draw a blue line.
   - **Trackbars**: Adjust HSV thresholds, blending alpha (0-100), and rotation angle (0-360) to see real-time effects.
   - Press 'q' to quit and save the output video.

## Code Explanation

### Imports
- `cv2`: OpenCV library for computer vision tasks.
- `np`: NumPy for numerical operations.
- `datetime`: For adding current date and time to the video feed.

### Mouse Callback Function (`draw_shapes`)
- Defines interactive drawing:
  - **Double Left-Click**: Draws a filled red circle (radius 50) at the click position.
  - **Right-Click**: Draws a filled green triangle using predefined points.
  - **Left-Click Drag**: Draws a blue line between the last and current mouse positions, updating the last point.

### Trackbar Callback Function (`nothing`)
- A placeholder function required for creating trackbars, as OpenCV expects a callback even if no action is needed.

### Image Loading and Preprocessing
- Loads `Do_It.jpg` and `Skyscraper.jpg`, exiting if either fails.
- **Channel Manipulation**: Splits `Skyscraper.jpg` into B, G, R channels, increases blue channel intensity by 50, and merges back.
- **Resizing**: Resizes images to match the video frame (640x480) or a smaller size (200x300) for overlay.

### Video Capture Setup
- Initializes the camera with a resolution of 640x480 and brightness of 150.
- Exits if the camera fails to open.

### Window Creation
- Creates four OpenCV windows for different visualizations:
  - `Video Feed`: Main interactive window.
  - `Sky Effects`: Shows processed `Do_It.jpg`.
  - `Skyscraper Effects`: Shows processed `Skyscraper.jpg`.
  - `Bitwise Operations`: Displays bitwise operation results.

### Main Loop
- **Video Feed Window**:
  - Overlays the current date and time (e.g., "2025-10-04 16:10" CEST) in yellow.
  - Draws a blue rectangle and a yellow ellipse as geometric shapes.
  - Applies HSV-based object detection with a green bounding box and label.
  - Rotates the `Skyscraper.jpg` overlay based on the `Rotation Angle` trackbar and blends it with the video feed using `Alpha` weighting.
- **Sky Effects Window**:
  - Converts `Do_It.jpg` to grayscale, applies Canny edge detection, inverts colors, and blends them for a unique effect.
- **Skyscraper Effects Window**:
  - Applies Gaussian blur and increases contrast on `Skyscraper.jpg` for a smoothed, enhanced look.
- **Bitwise Operations Window**:
  - Performs AND, OR, XOR, and NOT operations on resized `Do_It.jpg` and `Skyscraper.jpg`, stacking the results horizontally.
- Saves the `Video Feed` to `output_video.avi` and exits on 'q' key press.

### Resource Cleanup
- Releases the camera and video writer, then destroys all windows.

## Notes
- The script assumes a working camera. If no camera is detected, it will exit.
- Adjust trackbars to experiment with different effects in real-time.
- Ensure images are in the correct directory or provide full paths to avoid loading errors.

In [1]:
import cv2
import numpy as np
from datetime import datetime


In [ ]:

# Mouse callback function to draw a circle on double click
def draw_circle(event, x, y, flags, param):
    global frame  # Use the global frame to draw on it
    if event == cv2.EVENT_LBUTTONDBLCLK:
        cv2.circle(frame, (x, y), 50, (0, 0, 255), -1)  # Draw a red circle

# Trackbar callback (does nothing, required for createTrackbar)
def nothing(x):
    pass

# Load the provided images
sky = cv2.imread('Do_It.jpg')
if sky is None:
    print("Error: Could not load Do_It.jpg")
    exit()

skyscraper = cv2.imread('Skyscraper.jpg')
if skyscraper is None:
    print("Error: Could not load Skyscraper.jpg")
    exit()

# Demonstrate cv.split and cv.merge on skyscraper image
b, g, r = cv2.split(skyscraper)
# Modify one channel (e.g., set blue to zero for demonstration)
b = np.zeros_like(b)
modified_skyscraper = cv2.merge([b, g, r])

# Resize skyscraper for overlay as ROI (small size)
small_skyscraper = cv2.resize(modified_skyscraper, (200, 300))

# Open video capture from camera
cap = cv2.VideoCapture(0)

# Set camera parameters (resolution)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera")
    exit()

# Create a named window for display and controls
cv2.namedWindow('Video Feed')

# Set mouse callback for handling mouse events
cv2.setMouseCallback('Video Feed', draw_circle)

# Create trackbars for HSV thresholds (for object detection)
cv2.createTrackbar('H Min', 'Video Feed', 0, 179, nothing)
cv2.createTrackbar('S Min', 'Video Feed', 0, 255, nothing)
cv2.createTrackbar('V Min', 'Video Feed', 0, 255, nothing)
cv2.createTrackbar('H Max', 'Video Feed', 179, 179, nothing)
cv2.createTrackbar('S Max', 'Video Feed', 255, 255, nothing)
cv2.createTrackbar('V Max', 'Video Feed', 255, 255, nothing)

# Create trackbar for blending alpha (for cv.addWeighted)
cv2.createTrackbar('Alpha (0-100)', 'Video Feed', 50, 100, nothing)

# Prepare video writer to save the output
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (640, 480))

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break

    # Get current date and time, display on video
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cv2.putText(frame, current_time, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

    # Draw a geometric shape (e.g., a rectangle) on the frame
    cv2.rectangle(frame, (50, 50), (150, 150), (255, 0, 0), 3)  # Blue rectangle

    # Get trackbar positions for HSV
    
    h_min = cv2.getTrackbarPos('H Min', 'Video Feed')
    s_min = cv2.getTrackbarPos('S Min', 'Video Feed')
    v_min = cv2.getTrackbarPos('V Min', 'Video Feed')
    h_max = cv2.getTrackbarPos('H Max', 'Video Feed')
    s_max = cv2.getTrackbarPos('S Max', 'Video Feed')
    v_max = cv2.getTrackbarPos('V Max', 'Video Feed')

    # Convert frame to HSV for object detection/tracking
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_bound = np.array([h_min, s_min, v_min])
    upper_bound = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(hsv_frame, lower_bound, upper_bound)

    # Bitwise operations: Apply mask to frame (bitwise AND), invert mask (bitwise NOT)
    result = cv2.bitwise_and(frame, frame, mask=mask)
    inv_mask = cv2.bitwise_not(mask)

    # Find contours for object tracking (simple bounding box on largest object)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest_contour) > 500:  # Minimum area threshold
            x, y, w, h = cv2.boundingRect(largest_contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green bounding box

    # Resize sky image to match frame size
    sky_resized = cv2.resize(sky, (frame.shape[1], frame.shape[0]))

    # Get alpha from trackbar for blending
    alpha = cv2.getTrackbarPos('Alpha (0-100)', 'Video Feed') / 100.0

    # Blend the frame with the sky image using cv.addWeighted
    blended = cv2.addWeighted(frame, alpha, sky_resized, 1 - alpha, 0)

    # Use ROI to overlay the small skyscraper image on the blended frame
    roi_height, roi_width = small_skyscraper.shape[:2]
    if roi_height <= blended.shape[0] and roi_width <= blended.shape[1]:
        roi = blended[0:roi_height, 0:roi_width]
        # Blend skyscraper into ROI using cv.add
        overlaid_roi = cv2.add(roi, small_skyscraper)  # Simple add (could use addWeighted for better effect)
        blended[0:roi_height, 0:roi_width] = overlaid_roi

    # Display the final frame
    cv2.imshow('Video Feed', blended)

    # Write the frame to the output video
    out.write(blended)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2570: error: (-27:Null pointer) NULL window: 'Video Feed' in function 'cvGetTrackbarPos'


In [ ]:
# Mouse callback function to draw shapes based on different events
def draw_shapes(event, x, y, flags, param):
    global frame  # Use global frame for drawing
    if event == cv2.EVENT_LBUTTONDBLCLK:  # Double click to draw a red circle
        cv2.circle(frame, (x, y), 50, (0, 0, 255), -1)
    elif event == cv2.EVENT_RBUTTONDOWN:  # Right click to draw a green triangle
        pts = np.array([[x-50, y+50], [x, y-50], [x+50, y+50]], np.int32)
        cv2.fillPoly(frame, [pts], (0, 255, 0))
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:  # Drag to draw a blue line
        cv2.line(frame, param[0], (x, y), (255, 0, 0), 2)
        param[0] = (x, y)  # Update last point

# Trackbar callback (does nothing, required for createTrackbar)
def nothing(x):
    pass

# Load the provided images
sky = cv2.imread('Do_It.jpg')
if sky is None:
    print("Error: Could not load Do_It.jpg")
    exit()

skyscraper = cv2.imread('Skyscraper.jpg')
if skyscraper is None:
    print("Error: Could not load Skyscraper.jpg")
    exit()

# Split and merge with channel manipulation
b, g, r = cv2.split(skyscraper)
b = cv2.add(b, 50)  # Increase blue channel intensity
modified_skyscraper = cv2.merge([b, g, r])

# Resize images for different windows and bitwise operations
small_skyscraper = cv2.resize(modified_skyscraper, (200, 300))
sky_resized_base = cv2.resize(sky, (640, 480))  # Match video frame size
skyscraper_resized = cv2.resize(skyscraper, (640, 480))  # Match video frame size for bitwise ops

# Open video capture from camera
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_BRIGHTNESS, 150)

if not cap.isOpened():
    print("Error: Could not open camera")
    exit()

# Create multiple named windows
cv2.namedWindow('Video Feed')
cv2.namedWindow('Sky Effects')
cv2.namedWindow('Skyscraper Effects')
cv2.namedWindow('Bitwise Operations')

# Set mouse callback with last point for dragging
last_point = [(0, 0)]
cv2.setMouseCallback('Video Feed', draw_shapes, last_point)

# Create trackbars for controls
cv2.createTrackbar('H Min', 'Video Feed', 0, 179, nothing)
cv2.createTrackbar('S Min', 'Video Feed', 0, 255, nothing)
cv2.createTrackbar('V Min', 'Video Feed', 0, 255, nothing)
cv2.createTrackbar('H Max', 'Video Feed', 179, 179, nothing)
cv2.createTrackbar('S Max', 'Video Feed', 255, 255, nothing)
cv2.createTrackbar('V Max', 'Video Feed', 255, 255, nothing)
cv2.createTrackbar('Alpha (0-100)', 'Video Feed', 50, 100, nothing)
cv2.createTrackbar('Rotation Angle', 'Video Feed', 0, 360, nothing)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (640, 480))

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break

    # Video Feed Window: Main interactive view
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cv2.putText(frame, current_time, (10, 50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv2.rectangle(frame, (50, 200), (150, 300), (255, 0, 0), 3)
    cv2.ellipse(frame, (400, 100), (50, 30), 0, 0, 360, (0, 255, 255), 2)

    h_min = cv2.getTrackbarPos('H Min', 'Video Feed')
    s_min = cv2.getTrackbarPos('S Min', 'Video Feed')
    v_min = cv2.getTrackbarPos('V Min', 'Video Feed')
    h_max = cv2.getTrackbarPos('H Max', 'Video Feed')
    s_max = cv2.getTrackbarPos('S Max', 'Video Feed')
    v_max = cv2.getTrackbarPos('V Max', 'Video Feed')
    alpha = cv2.getTrackbarPos('Alpha (0-100)', 'Video Feed') / 100.0
    angle = cv2.getTrackbarPos('Rotation Angle', 'Video Feed')

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_bound = np.array([h_min, s_min, v_min])
    upper_bound = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(hsv_frame, lower_bound, upper_bound)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest_contour) > 500:
            x, y, w, h = cv2.boundingRect(largest_contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, 'Object', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    M = cv2.getRotationMatrix2D((small_skyscraper.shape[1]/2, small_skyscraper.shape[0]/2), angle, 1)
    rotated_skyscraper = cv2.warpAffine(small_skyscraper, M, (small_skyscraper.shape[1], small_skyscraper.shape[0]))
    blended = cv2.addWeighted(frame, alpha, sky_resized_base, 1 - alpha, 0)
    roi_height, roi_width = rotated_skyscraper.shape[:2]
    if roi_height <= blended.shape[0] and roi_width <= blended.shape[1]:
        roi = blended[0:roi_height, 0:roi_width]
        overlaid_roi = cv2.addWeighted(roi, 0.7, rotated_skyscraper, 0.3, 0)
        blended[0:roi_height, 0:roi_width] = overlaid_roi
    cv2.imshow('Video Feed', blended)
    out.write(blended)

    # Sky Effects Window: Apply edge detection and color inversion with channel matching
    sky_gray = cv2.cvtColor(sky, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(sky_gray, 100, 200)
    inverted_sky = cv2.bitwise_not(sky)
    edges_3channel = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    sky_effect = cv2.addWeighted(edges_3channel, 0.5, inverted_sky, 0.5, 0)
    cv2.imshow('Sky Effects', sky_effect)

    # Skyscraper Effects Window: Apply blur and contrast adjustment
    blurred_skyscraper = cv2.GaussianBlur(skyscraper, (15, 15), 0)
    alpha_contrast = 1.5  # Increase contrast
    beta_contrast = 0
    contrasted_skyscraper = cv2.convertScaleAbs(blurred_skyscraper, alpha=alpha_contrast, beta=beta_contrast)
    cv2.imshow('Skyscraper Effects', contrasted_skyscraper)

    # Bitwise Operations Window: Showcase AND, OR, XOR (with resized images)
    bitwise_and = cv2.bitwise_and(sky_resized_base, skyscraper_resized)
    bitwise_or = cv2.bitwise_or(sky_resized_base, skyscraper_resized)
    bitwise_xor = cv2.bitwise_xor(sky_resized_base, skyscraper_resized)
    bitwise_not_sky = cv2.bitwise_not(sky_resized_base)
    stacked_bitwise = np.hstack((bitwise_and, bitwise_or, bitwise_xor, bitwise_not_sky))
    cv2.imshow('Bitwise Operations', stacked_bitwise)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2570: error: (-27:Null pointer) NULL window: 'Video Feed' in function 'cvGetTrackbarPos'
